In [1]:
import pandas as pd
import chess

In [84]:
import numpy as np

In [143]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset

In [149]:
import torch.nn.functional as F 

In [115]:
from sklearn.model_selection import train_test_split

In [199]:
device = torch.device(0 if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [121]:
df = pd.read_csv('./dataset.csv')

In [122]:
df.head()

,fen,score
0,rnbqkb1r/pppppppp/5n2/8/4P3/8/PPPP1PPP/RNBQKBN...,172
1,rnbqkb1r/pp2pp1p/1n1p2p1/8/2P5/2N2N2/PP1P1PPP/...,144
2,rnbqkb1r/pp2pp1p/1n1p2p1/8/2PP4/2N2N2/PP3PPP/R...,123
3,r2q1rk1/pp2ppbp/1nnp2p1/8/2PP4/2N1BB2/PP3PPP/R...,-169
4,r4rk1/p4pbp/1q4p1/2Rpp3/3P4/1P1Q4/P3N1PP/5RK1 ...,-233


In [182]:
class chessPositions(Dataset):
    def __init__(self, data):
        self.data = data
        self.piece2Idx = {
            'r': 1, 'b': 2, 'k': 3,
            'q': 4, 'n': 5, 'p': 6,
            'P': 7, 'R': 8, 'N': 9,
            'B': 10, 'K': 11, 'Q': 12
        }
    
    def __getitem__(self,idx):
        fen, y = self.data[idx]
        vector, features = self.genX(fen)
        vector = torch.tensor(vector, dtype=torch.float64, device=device)
        features = torch.tensor(features, dtype=torch.float64, device=device)
        y = torch.tensor(y, dtype=torch.float64, device=device)
        return vector, features, y
    
    def __len__(self):
        return self.data.shape[0]
    
    def genX(self, fen):
        board = chess.Board(fen)
        vector = self.getBoardRep(board)
        handCrafted = self.handCraftedFeatures(board)

        return vector, handCrafted

    def getBoardRep(self, board):
        pieces = board.piece_map()
        vector = [0 for _ in range(64)]

        for (idx, piece) in pieces.items():
            symbol = piece.symbol()
            num = self.piece2Idx[symbol]
            vector[idx] = num

        return np.array(vector)


    def getMaterial(self, board):
        wp = len(board.pieces(chess.PAWN, chess.WHITE))
        bp = len(board.pieces(chess.PAWN, chess.BLACK))
        wn = len(board.pieces(chess.KNIGHT, chess.WHITE))
        bn = len(board.pieces(chess.KNIGHT, chess.BLACK))
        wb = len(board.pieces(chess.BISHOP, chess.WHITE))
        bb = len(board.pieces(chess.BISHOP, chess.BLACK))
        wr = len(board.pieces(chess.ROOK, chess.WHITE))
        br = len(board.pieces(chess.ROOK, chess.BLACK))
        wq = len(board.pieces(chess.QUEEN, chess.WHITE))
        bq = len(board.pieces(chess.QUEEN, chess.BLACK))

        total_material = bp + bn + bb + br + bq + wp + wn + wb + wr + wq
        return total_material

    def countAttacked(self, board):
        pieces = board.piece_map()
        idxs = list(pieces.keys())

        white_attacked = 0
        black_attacked = 0

        for idx in idxs:
            square = chess.SQUARE_NAMES[idx]

            #True - white, false - black
            attack_color = chess.WHITE if not board.color_at(idx) else chess.BLACK

            num_attackers = len(board.attackers(attack_color, idx))

            if num_attackers > 0:
                if not attack_color:
                    white_attacked += 1
                else:
                    black_attacked += 1

        return black_attacked - white_attacked

    def handCraftedFeatures(self, board):
        material = getMaterial(board)
        attacked = countAttacked(board)
        return np.array([material, attacked])

In [154]:
class fusionModel(nn.Module):
    
    def __init__(self):
        super(fusionModel, self).__init__()
        
        self.v1 = nn.Linear(64, 1024)
        self.v2 = nn.Linear(1024, 1024)
        
        self.f1 = nn.Linear(2, 32)
        self.f2 = nn.Linear(32, 32)
        
        self.fc1 = nn.Linear(1056, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 1)
        
    def forward(self, vector, features):
        vector = F.relu(self.v1(vector))
        vector = F.relu(self.v2(vector))
        
        features = F.relu(self.f1(features))
        features = F.relu(self.f2(features))
        
        x = torch.cat((vector, features), 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [120]:
df.columns

Index(['fen', ' score'], dtype='object')

In [136]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [183]:
trainSet = DataLoader(
    chessPositions(train.values), 
    num_workers=4, 
    batch_size=32
)
testSet = DataLoader(
    chessPositions(test.values), 
    num_workers=4,
    batch_size=32
)

In [200]:
model = fusionModel()
model.to(device)

RuntimeError: CUDA error: an illegal memory access was encountered

In [163]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [181]:
for (vector, features, y) in trainSet:
    optimizer.zero_grad()
    vector = vector.to(device)
    features = features.float().to(device)
    y = y.float().to(device)
    
    y_pred = model(vector, features)
    loss = criterion(y_pred, y)
    print(loss.item())
    loss.backward()
    optimizer.step()
    

RuntimeError: CUDA error: an illegal memory access was encountered